# Sentiment Analysis of Tweets
Play around with sentiment analysis of tweets to formulate method to engineer features.

## Import twitter data

In [1]:
# Replace '&amp;' with 'and'
def replaceAmp(text):
    """
    Takes in string and returns string with '&amp;' replaced with 'and'
    """
    return text.replace('&amp;', 'and')

### Import data from tweets_search

In [2]:
import boto3
from boto3.dynamodb.conditions import Key, Attr

TABLENAME = 'tweets_search'

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table(TABLENAME)

In [3]:
print(TABLENAME + " table item count:")
print(table.item_count)

tweets_search table item count:
635


In [4]:
tweets_search = {}
response = table.scan()
data = response['Items']
for d in data:
    text, sarcastic, id = d["text"], d["sarcastic"], d["id"]
    text = replaceAmp(text)
    if text not in tweets_search and sarcastic:
        tweets_search[text] = (sarcastic, id)
    
while response.get('LastEvaluatedKey'):
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data = response['Items']
    for d in data:
        text, sarcastic, id = d["text"], d["sarcastic"], d["id"]
        text = replaceAmp(text)
        if text not in tweets_search and sarcastic:
            tweets_search[text] = (sarcastic, id)

print("count: " + str(len(tweets_search)))

count: 503


### Import data from tweets_sarcastic

In [5]:
TABLENAME = 'tweets_sarcastic'

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table(TABLENAME)

In [6]:
print(TABLENAME + " table item count:")
print(table.item_count)

tweets_sarcastic table item count:
197


In [7]:
tweets_sarcastic = {}
response = table.scan()
data = response['Items']
for d in data:
    text, sarcastic, id = d["text"], d["sarcastic"], d["id"]
    text = replaceAmp(text)
    if text not in tweets_sarcastic and sarcastic:
        tweets_sarcastic[text] = (sarcastic, id)
    
while response.get('LastEvaluatedKey'):
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data = response['Items']
    for d in data:
        text, sarcastic, id = d["text"], d["sarcastic"], d["id"]
        text = replaceAmp(text)
        if text not in tweets_sarcastic and sarcastic:
            tweets_sarcastic[text] = (sarcastic, id)

print("count: " + str(len(tweets_sarcastic)))

count: 172


### Combine two datasets

In [8]:
tweets = {}
for k, v in tweets_search.items():
    tweets[k] = v
for k, v in tweets_sarcastic.items():
    if k not in tweets:
        tweets[k] = v
print("count: " + str(len(tweets)))

count: 670


In [9]:
count = 0
for k, v in tweets.items():
    if '&amp;' in k:
        print(k)

### Convert dictionary to pandas dataframe

In [10]:
import pandas as pd
df = pd.DataFrame(data = tweets)

ImportError: No module named 'pandas'

## Sentiment Analysis